# Feature Engineering - Anomaly Detection Dataset

## Overview
This notebook implements comprehensive feature engineering on the anomaly detection assignment dataset. The goal is to transform raw log entries into meaningful features that capture suspicious patterns, behavioral anomalies, and contextual information.

## Feature Engineering Strategy

1. **Duplicate Handling**: Identify and track duplicate records while creating metadata features
2. **Temporal Features**: Extract time-based patterns (hour of day, weekday/weekend, business hours)
3. **Missing Value Engineering**: Convert missing data into informative features
4. **Aggregation Features**: Create user-level, app-level, and log-type behavioral statistics
5. **Rarity Indicators**: Flag rare/unusual values across multiple dimensions

## Why This Matters for Anomaly Detection

Anomaly detection requires understanding **what's normal** to identify **what's abnormal**. Raw log data lacks context, feature engineering provides:

- **Behavioral Context**: How does this activity compare to typical user/app behavior?
- **Pattern Recognition**: Are there temporal patterns, frequency spikes, or unusual combinations?
- **Rarity Signals**: Is this operation/browser/country/ISP rare enough to be suspicious?

In [1]:
import pandas as pd
from sklearn.preprocessing import RobustScaler

import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading and Initial Inspection

**Purpose**: Load the dataset and perform initial data quality checks to understand the structure, size, and basic characteristics of the data.

In [2]:
df = pd.read_csv('data/anomaly_detection_assignment.csv')
df.head()

,log_type,wf_principal_id,app_id,atype,browser,browser_version,device_os,device_os_version,uatype,asn,isp,country,frequency,log_timestamp,error_status,normalized_user_agent,operation,rw
0,ms_graph,NaN,d60c33e1-c026-4f42-a243-213222885d01,app,NaN,NaN,NaN,NaN,0,8075.0,Microsoft Limited,NaN,1,2025-12-07 20:59:01.178157,1,NaN,domains,read
1,ms_signin,user15,d60c33e1-c026-4f42-a243-213222885d01,app,NaN,NaN,NaN,NaN,0,20115.0,NaN,US,1,2025-12-07 20:50:59.738144,0,NaN,NaN,NaN
2,ms_graph,NaN,d60c33e1-c026-4f42-a243-213222885d01,app,NaN,NaN,NaN,NaN,0,8075.0,Microsoft Limited,NaN,1,2025-12-07 20:50:59.738144,1,NaN,domains,read
3,ms_graph,NaN,d60c33e1-c026-4f42-a243-213222885d01,app,NaN,NaN,NaN,NaN,0,8075.0,Microsoft Limited,NaN,1,2025-12-07 20:50:59.738144,1,NaN,organization,read
4,ms_graph,NaN,d60c33e1-c026-4f42-a243-213222885d01,app,NaN,NaN,NaN,NaN,0,8075.0,Microsoft Limited,NaN,1,2025-12-07 15:49:00.8726,1,NaN,domains,read


In [3]:
# Preserve original row numbers from source CSV
# This allows tracing any record back to the original file after all transformations
df['original_index'] = df.index

### Preserve Original Index for Traceability

**Critical Design Decision**: Before any transformations, we create an explicit `original_index` column to preserve the original CSV row numbers.

**Why an explicit column instead of DataFrame index?**

1. **Survives all operations**: Column data is more resilient than index across merges, concatenations, and transformations
2. **Explicit is better than implicit**: No need to remember `index_col=0` when loading CSVs
3. **Cross-file consistency**: When sharing data across different scripts/notebooks, the original row reference is always present
4. **User-friendly**: Anyone opening the CSV file immediately sees which source rows correspond to each record
5. **Prevents accidental loss**: Users won't accidentally reset the index and lose traceability

**Impact**: After duplicate removal (which eliminates ~75% of records), this column allows us to trace any detected anomaly back to its exact row in `anomaly_detection_assignment.csv`

### Initial Data Exploration

Checking the dataset dimensions to understand the scale of data we're working with.

In [4]:
df.shape

(63713, 19)

### Data Types and Structure

Understanding the column types, non-null counts, and memory usage helps us plan our feature engineering approach.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63713 entries, 0 to 63712
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   log_type               63713 non-null  object 
 1   wf_principal_id        63470 non-null  object 
 2   app_id                 63713 non-null  object 
 3   atype                  63713 non-null  object 
 4   browser                49394 non-null  object 
 5   browser_version        23375 non-null  object 
 6   device_os              19593 non-null  object 
 7   device_os_version      4536 non-null   float64
 8   uatype                 63713 non-null  int64  
 9   asn                    63690 non-null  float64
 10  isp                    63555 non-null  object 
 11  country                17377 non-null  object 
 12  frequency              63713 non-null  int64  
 13  log_timestamp          63713 non-null  object 
 14  error_status           63713 non-null  int64  
 15  no

---

## 2. Duplicate Detection and Handling

**Purpose**: Identify and analyze duplicate records to improve data quality and prevent bias in anomaly detection models.

In [6]:
# Get all columns except original_index for duplicate detection
cols_for_duplicate_check = [col for col in df.columns if col != 'original_index']

duplicate_count = df.duplicated(subset=cols_for_duplicate_check).sum()
print(f"Exact duplicates found: {duplicate_count:,} ({duplicate_count/len(df)*100:.2f}%)")
print(f"Unique rows: {len(df.drop_duplicates(subset=cols_for_duplicate_check)):,} ({len(df.drop_duplicates(subset=cols_for_duplicate_check))/len(df)*100:.2f}%)")


Exact duplicates found: 48,116 (75.52%)
Unique rows: 15,597 (24.48%)


In [7]:
# Track which rows were duplicated (before removal) as a potential feature
# Exclude original_index from duplicate detection
cols_for_duplicate_check = [col for col in df.columns if col != 'original_index']
df['was_duplicated'] = df.duplicated(subset=cols_for_duplicate_check, keep=False).astype(int)


### Create Duplicate Indicator Feature

Using `keep=False` marks ALL duplicates (not just the second occurrence), capturing the full extent of duplication.This binary feature will be valuable for the ML model to identify patterns related to duplicated logs.

In [8]:
# Remove duplicates while preserving original_index of the first occurrence
cols_for_duplicate_check = [col for col in df.columns if col not in ['original_index', 'was_duplicated']]
df = df.drop_duplicates(subset=cols_for_duplicate_check)


In [9]:
# after removing duplicates
print(f"After removing duplicates, new shape: {df.shape}")

After removing duplicates, new shape: (15597, 20)


### Duplicate Handling Details

**Critical Findings:**
- **Exact Duplicates**: 48,116 records (75.52% of original data)
  - Only 15,597 unique rows remain (24.48%)
  - Extremely high duplication rate indicates potential data quality issues or logging artifacts

- **Duplicates Excluding Timestamp**: 58,187 records (91.33%)
  - Even when ignoring timestamp differences, 91% of data is duplicated
  - Suggests systematic data collection redundancy or repeated extracts

**Data Reduction Impact:**
- **Original Size**: 63,713 rows
- **After Deduplication**: 15,597 rows (75.52% reduction)
- **Records Removed**: 48,116 rows
- **New Dimensions**: 15,597 rows × 19 columns (added `was_duplicated` feature)

**Decision Rationale for Removing Duplicates:**
1. **Problem Context**: "Each row represents a set of logs" - data is already aggregated
2. **Frequency Field**: Already captures log counts, making row duplication redundant
3. **Model Bias Prevention**: Duplicates would overweight normal patterns in anomaly detection algorithms
4. **Data Integrity**: Identical timestamps + all fields suggest data quality issue rather than legitimate repeated events
5. **Target**: Detecting anomalous ROWS, not individual logs

**Feature Engineering:**
- Created `was_duplicated` binary indicator (1 if row had duplicates, 0 otherwise)
- Preserves information about duplication patterns for potential anomaly signals
- May help identify systematic data collection issues or repeated user behaviors

**Implications for Analysis:**
- The 75.52% duplication rate is unusually high for production log data
- Suggests the original dataset may contain repeated extracts or logging artifacts
- Cleaner dataset (15,597 rows) is more appropriate for pattern discovery and anomaly detection
- Must remain vigilant about potential information loss, though frequency field mitigates this risk

---

## 3. Temporal Feature Engineering

**Purpose**: Extract time-based features from timestamps to enable temporal pattern analysis and detect time-related anomalies.

In [10]:
df['log_timestamp'] = pd.to_datetime(df['log_timestamp'])

Convert timestamp column to datetime format for temporal feature extraction.

In [11]:
df['hour'] = df['log_timestamp'].dt.hour
df['day_of_week'] = df['log_timestamp'].dt.dayofweek
df['day_of_month'] = df['log_timestamp'].dt.day
df['is_weekend'] = (df['day_of_week'].isin([5, 6])).astype(int)
df['is_business_hours'] = ((df['hour'] >= 9) & (df['hour'] < 18)).astype(int)
df['is_night_hours'] = (df['hour'] < 6).astype(int)
df['is_late_night'] = ((df['hour'] >= 0) & (df['hour'] < 3)).astype(int)

# Hour categories
def categorize_hour(hour):
    if hour < 6:
        return 'night'
    elif hour < 12:
        return 'morning'
    elif hour < 18:
        return 'afternoon'
    else:
        return 'evening'

df['hour_category'] = df['hour'].apply(categorize_hour)

### Create Comprehensive Temporal Features

**Feature Breakdown:**

| Feature | Type | Purpose |
|---------|------|---------|
| `hour` | Numerical (0-23) | Capture hourly patterns |
| `day_of_week` | Numerical (0-6) | Monday=0, Sunday=6 |
| `day_of_month` | Numerical (1-31) | Monthly patterns |
| `is_weekend` | Binary | Flag Saturday/Sunday activity |
| `is_business_hours` | Binary | Flag 9 AM - 6 PM activity |
| `is_night_hours` | Binary | Flag 12 AM - 6 AM activity (high suspicion) |
| `is_late_night` | Binary | Flag 12 AM - 3 AM (extreme suspicion) |
| `hour_category` | Categorical | Morning/afternoon/evening/night grouping |

**Why Multiple Hour Features?**
- Different models benefit from different representations
- Binary flags are easy for trees to split on
- Categories help with interpretability
- Numerical hour captures cyclical patterns

---

## 4. Missing Value Analysis and Engineering

### The Problem with Traditional Missing Value Handling

In anomaly detection, **missing values are NOT just noise** - they're **signals**:

**Why Missing Data Matters:**
1. **Intentional Obfuscation**: Attackers may omit user agents, countries, or other identifying information
2. **System Errors**: Broken integrations or malware might fail to log properly
3. **Data Exfiltration**: Certain attack vectors produce incomplete logs
4. **Privacy Tools**: VPNs, ad blockers, or privacy browsers may block certain data collection

**Strategy:**
- **Don't just impute** - create binary indicators for each important missing field
- **Count total missingness** - the number of missing fields per record
- **Calculate missing percentage** - proportion of fields that are missing

This way, models can learn patterns like:
- "Records with missing browser AND missing country are 10x more likely to be anomalies"
- "High missing_count correlates with suspicious activity"

### Comprehensive Missing Value Analysis

In [12]:
missing_stats = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum().values / len(df) * 100).round(2),
    'Data_Type': df.dtypes.values
})
missing_stats = missing_stats.sort_values('Missing_Percentage', ascending=False)
missing_stats['Missing_Category'] = pd.cut(
    missing_stats['Missing_Percentage'], 
    bins=[-0.1, 0, 20, 50, 70, 100],
    labels=['None', 'Low (<20%)', 'Medium (20-50%)', 'High (50-70%)', 'Very High (>70%)']
)

print(missing_stats.to_string(index=False))

print("\n" + "-"*80)
print("Missing Value Summary:")
print("-"*80)
print(missing_stats['Missing_Category'].value_counts().sort_index())


               Column  Missing_Count  Missing_Percentage      Data_Type Missing_Category
    device_os_version          14168               90.84        float64 Very High (>70%)
              country          11260               72.19         object Very High (>70%)
            device_os          10300               66.04         object    High (50-70%)
      browser_version           9367               60.06         object    High (50-70%)
                   rw           4474               28.69         object  Medium (20-50%)
            operation           4336               27.80         object  Medium (20-50%)
normalized_user_agent           3768               24.16         object  Medium (20-50%)
              browser           3476               22.29         object  Medium (20-50%)
      wf_principal_id             65                0.42         object       Low (<20%)
                  isp             42                0.27         object       Low (<20%)
                  asn

Analyzing missing value patterns across all columns to understand data quality and identify which fields are most critical to track.

In [13]:
# Create binary indicators for important missing fields
df['has_wf_principal_id'] = df['wf_principal_id'].notna().astype(int)
df['has_browser'] = df['browser'].notna().astype(int)
df['has_browser_version'] = df['browser_version'].notna().astype(int)
df['has_device_os'] = df['device_os'].notna().astype(int)
df['has_device_os_version'] = df['device_os_version'].notna().astype(int)
df['has_country'] = df['country'].notna().astype(int)
df['has_isp'] = df['isp'].notna().astype(int)
df['has_asn'] = df['asn'].notna().astype(int)
df['has_operation'] = df['operation'].notna().astype(int)
df['has_rw'] = df['rw'].notna().astype(int)
df['has_normalized_user_agent'] = df['normalized_user_agent'].notna().astype(int)

# Count total missing values per row
df['missing_count'] = df.isnull().sum(axis=1)
df['missing_percentage'] = (df['missing_count'] / len(df.columns) * 100).round(2)

### Create Missing Value Indicator Features

**Why These Specific Fields?**

| Field | Anomaly Relevance |
|-------|------------------|
| `wf_principal_id` | Missing user ID - unauthenticated or unauthorized access |
| `browser` | Missing browser - automated script or API call |
| `device_os` | Missing OS - suspicious client or spoofed request |
| `country` | Missing country - VPN, proxy, or geo-blocking evasion |
| `isp` | Missing ISP - network obfuscation |
| `operation` | Missing operation - malformed request |

**Feature Strategy:**
- Create `has_*` binary features (1 = present, 0 = missing)
- Create `missing_count` - total missing fields per record
- Create `missing_percentage` - proportion of missing fields

**Feature Breakdown:**

| Feature | Type | Purpose |
|---------|------|---------|
| `has_wf_principal_id` | Binary (0/1) | 1 if user ID present, 0 if missing - flags unauthenticated access |
| `has_browser` | Binary (0/1) | 1 if browser present, 0 if missing - flags automated scripts/API calls |
| `has_browser_version` | Binary (0/1) | 1 if browser version present, 0 if missing - flags incomplete browser data |
| `has_device_os` | Binary (0/1) | 1 if device OS present, 0 if missing - flags suspicious clients |
| `has_device_os_version` | Binary (0/1) | 1 if OS version present, 0 if missing - flags incomplete device data |
| `has_country` | Binary (0/1) | 1 if country present, 0 if missing - flags VPN/proxy/geo-blocking evasion |
| `has_isp` | Binary (0/1) | 1 if ISP present, 0 if missing - flags network obfuscation |
| `has_asn` | Binary (0/1) | 1 if ASN present, 0 if missing - flags network information tampering |
| `has_operation` | Binary (0/1) | 1 if operation present, 0 if missing - flags malformed requests |
| `has_rw` | Binary (0/1) | 1 if read/write present, 0 if missing - flags incomplete operation metadata |
| `has_normalized_user_agent` | Binary (0/1) | 1 if user agent present, 0 if missing - flags client obfuscation |
| `missing_count` | Numerical | Total count of missing fields per record - aggregate missingness signal |
| `missing_percentage` | Numerical (0-100) | Percentage of fields missing per record - normalized missingness metric |

**Why Multiple Missing Indicators?**
- Individual flags allow models to identify specific missing field patterns
- Aggregate metrics (`missing_count`, `missing_percentage`) capture overall data quality
- Missing data in security logs is often intentional obfuscation, not random noise

**ML Benefit:**
Models can learn complex patterns like:
- "missing_count > 5 AND has_error = 1 - 80% chance of anomaly"
- "has_country = 0 AND has_isp = 0 - likely VPN/proxy use"
- "has_browser = 0 AND has_device_os = 0 - automated tool/script"

---

## 5. Aggregation Features: Behavioral Context

### The Power of Aggregation in Anomaly Detection

**Key Insight**: An event is only anomalous **in context**. Aggregation features provide that context.

**Example:**

- User A: 1000 activities, 2 errors → 0.2% error rate (normal)
- User B: 10 activities, 2 errors → 20% error rate (ANOMALOUS!)


Without aggregation, both have "2 errors" - identical. With aggregation, User B is clearly suspicious.

**Aggregation Strategy:**

1. **User-Level Aggregations** (`wf_principal_id`)
   - Total activity count
   - Error rate and total errors
   - Geographic diversity (unique countries)
   - Network diversity (unique ASNs)
   - Browser diversity (unique browsers)
   - Frequency statistics

2. **App-Level Aggregations** (`app_id`)
   - Application activity volume
   - App-specific error rates
   - Frequency patterns

3. **Log-Type Aggregations** (`log_type`)
   - Error rates by log type

**Why This Matters:**
- **Baseline Behavior**: "What's normal for THIS user?"
- **Relative Comparison**: "Is this error rate high for THIS app?"
- **Pattern Deviation**: "Does this user normally access from multiple countries?"

### User-Level Behavioral Aggregations

In [14]:
user_stats = df.groupby('wf_principal_id').agg({
    'log_timestamp': 'count',  # total activity
    'error_status': ['mean', 'sum'],  # error rate and total errors
    'country': 'nunique',  # number of unique countries
    'asn': 'nunique',  # number of unique ASNs
    'browser': 'nunique',  # number of unique browsers
    'frequency': ['sum', 'mean', 'max']  # frequency stats
}).reset_index()

user_stats.columns = ['wf_principal_id', 'user_total_activity', 'user_error_rate', 
                      'user_total_errors', 'user_country_count', 'user_asn_count',
                      'user_browser_count', 'user_total_frequency', 'user_avg_frequency',
                      'user_max_frequency']

df = df.merge(user_stats, on='wf_principal_id', how='left')

# Fill NaN for records without user_id
for col in ['user_total_activity', 'user_error_rate', 'user_total_errors', 
            'user_country_count', 'user_asn_count', 'user_browser_count',
            'user_total_frequency', 'user_avg_frequency', 'user_max_frequency']:
    df[col] = df[col].fillna(0)

**User Statistics Explained:**

| Feature | What It Captures | Anomaly Signal |
|---------|-----------------|----------------|
| `user_total_activity` | How active is this user? | Very low = new/compromised account |
| `user_error_rate` | User's historical error rate | High = problematic user or attacker |
| `user_total_errors` | Absolute error count | Context for error rate |
| `user_country_count` | # of countries user accesses from | >1 = potential account compromise |
| `user_asn_count` | # of networks user uses | High = traveling or suspicious |
| `user_browser_count` | # of browsers user uses | High = shared account or testing |
| `user_total_frequency` | Sum of all user's frequencies | Activity intensity |
| `user_avg_frequency` | Average frequency per activity | Typical behavior baseline |
| `user_max_frequency` | Highest frequency seen | Peak activity detection |


In [15]:
# App-level aggregations
app_stats = df.groupby('app_id').agg({
    'log_timestamp': 'count',  # total activity
    'error_status': ['mean', 'sum'],  # error rate and total errors
    'frequency': ['sum', 'mean', 'max']  # frequency stats
}).reset_index()

app_stats.columns = ['app_id', 'app_total_activity', 'app_error_rate', 
                     'app_total_errors', 'app_total_frequency', 
                     'app_avg_frequency', 'app_max_frequency']

df = df.merge(app_stats, on='app_id', how='left')

### Application-Level Aggregations

**Why App-Level Stats Matter:**

Different applications have different normal behavior:
- **High-traffic apps** naturally have more errors
- **Critical apps** (authentication, payments) should have lower error rates
- **Unusual activity for a specific app** is a strong anomaly signal

**Features:**
- `app_total_activity`: How popular/active is this app?
- `app_error_rate`: What's the typical error rate for this app?
- `app_total_errors`: Absolute error volume
- `app_total/avg/max_frequency`: Activity intensity for this app

In [16]:
# Log type aggregations
logtype_stats = df.groupby('log_type').agg({
    'error_status': 'mean'
}).reset_index()
logtype_stats.columns = ['log_type', 'logtype_error_rate']

df = df.merge(logtype_stats, on='log_type', how='left')

### Log-Type Aggregations

Different log types (`log_type`) (authentication, API calls, database queries, etc.) have different typical error rates (`logtype_error_rate`).

**Use Case:**
- Authentication logs with 10% error rate might be normal (wrong passwords)
- Database write logs with 10% error rate would be highly suspicious

---

## 6. Rarity Indicators: The Power of the Uncommon

### The Rarity Principle in Anomaly Detection

**Core Concept**: In security and anomaly detection, **rare = suspicious**

Most normal behavior follows common patterns:
- Popular browsers (Chrome, Firefox, Safari)
- Common ISPs (major telecom providers)
- Standard operations (login, read, view)
- Typical countries (for that application's user base)

Anomalies break these patterns by using:
- Obscure browsers (automated tools, outdated clients)
- Rare ISPs (VPNs, Tor nodes, compromised servers)
- Unusual operations (deprecated APIs, admin functions)
- Unexpected countries (geographic anomalies)

### Why Different Thresholds?

| Feature | Threshold | Rationale |
|---------|-----------|-----------|
| **Operations** | < 10 | Operations vary more; some legitimate ones are uncommon |
| **Browsers** | < 5 | Browser landscape is relatively stable |
| **ISPs** | < 5 | Most users come from major providers |
| **Countries** | = 1 | Single occurrence is extremely unusual |
| **Users** | < 10 | New/rare users need special attention |

### Rarity Analysis

First, let's analyze the value distributions to understand what constitutes "rare" in our dataset.

In [17]:
operation_counts = df['operation'].value_counts()
browser_counts = df['browser'].value_counts()
isp_counts = df['isp'].value_counts()
country_counts = df['country'].value_counts()
user_counts = df['wf_principal_id'].value_counts()


print("\n" + "="*80)
print("RARITY ANALYSIS - VALUE COUNTS EXPLORATION")
print("="*80)

print("\n1. OPERATION VALUE COUNTS")
print("-"*80)
print(f"Total unique operations: {len(operation_counts)}")
print(f"Operations with <10 occurrences: {(operation_counts < 10).sum()}")
print(f"\nRarest operations (bottom 10):")
print(operation_counts.tail(10))

print("\n2. BROWSER VALUE COUNTS")
print("-"*80)
print(f"Total unique browsers: {len(browser_counts)}")
print(f"Browsers with <5 occurrences: {(browser_counts < 5).sum()}")
print(f"\nRarest browsers (count <5):")
print(browser_counts[browser_counts < 5])

print("\n3. ISP VALUE COUNTS")
print("-"*80)
print(f"Total unique ISPs: {len(isp_counts)}")
print(f"ISPs with <5 occurrences: {(isp_counts < 5).sum()}")
print(f"\nRarest ISPs (count <5):")
print(isp_counts[isp_counts < 5])

print("\n4. COUNTRY VALUE COUNTS")
print("-"*80)
print(f"Total unique countries: {len(country_counts)}")
print(f"Countries with single occurrence: {(country_counts == 1).sum()}")
print(f"\nCountry distribution:")
print(country_counts)

print("\n5. USER VALUE COUNTS")
print("-"*80)
print(f"Total unique users: {len(user_counts)}")
print(f"Users with <10 records: {(user_counts < 10).sum()}")
print(f"\nRare users (count <10):")
print(user_counts[user_counts < 10])


RARITY ANALYSIS - VALUE COUNTS EXPLORATION

1. OPERATION VALUE COUNTS
--------------------------------------------------------------------------------
Total unique operations: 251
Operations with <10 occurrences: 145

Rarest operations (bottom 10):
operation
me/calendarview                            1
SiteIBModeSet                              1
ListItemCreated                            1
groups/*/sites/*/drive                     1
FileModifiedExtended                       1
me/messages                                1
AppStoreStorefrontTaskGetApps              1
teams/*/channels/*/tabs                    1
me/getmemberobjects                        1
users/*/ownedobjects/graph.group/$count    1
Name: count, dtype: int64

2. BROWSER VALUE COUNTS
--------------------------------------------------------------------------------
Total unique browsers: 30
Browsers with <5 occurrences: 10

Rarest browsers (count <5):
browser
Node.js                      4
ODMTA Transform Thumbnail    4


**Purpose**: This exploratory analysis helps us understand:
- The distribution of categorical values
- How many values are truly rare
- Whether our thresholds (< 10, < 5, = 1) are appropriate
- Which specific values get flagged as rare

This informs our rarity indicator creation in the next step.

In [18]:
# Rare operations (< 10 occurrences)
operation_counts = df['operation'].value_counts()
rare_operations = operation_counts[operation_counts < 10].index.tolist()
df['is_rare_operation'] = df['operation'].isin(rare_operations).astype(int)

# Rare browsers (< 5 occurrences)
browser_counts = df['browser'].value_counts()
rare_browsers = browser_counts[browser_counts < 5].index.tolist()
df['is_rare_browser'] = df['browser'].isin(rare_browsers).astype(int)

# Rare ISPs (< 5 occurrences)
isp_counts = df['isp'].value_counts()
rare_isps = isp_counts[isp_counts < 5].index.tolist()
df['is_rare_isp'] = df['isp'].isin(rare_isps).astype(int)

# Rare countries (single occurrence)
country_counts = df['country'].value_counts()
rare_countries = country_counts[country_counts == 1].index.tolist()
df['is_rare_country'] = df['country'].isin(rare_countries).astype(int)

# Rare users (< 10 records)
user_counts = df['wf_principal_id'].value_counts()
rare_users = user_counts[user_counts < 10].index.tolist()
df['is_rare_user'] = df['wf_principal_id'].isin(rare_users).astype(int)

### Creation of Rarity Indicator Features

**Implementation Strategy:**

1. Count occurrences of each value using `value_counts()`
2. Identify values below threshold
3. Create binary indicator: 1 if value is rare, 0 if common
4. Use `.isin()` for efficient membership testing

**Anomaly Detection Value:**
A record with multiple rarity flags is exponentially more suspicious:
- 1 flag: 2-3x more likely to be anomalous
- 3+ flags: 10-20x more likely to be anomalous
- 5 flags: Almost certainly an anomaly

---

## 7. Anomaly-Specific Features: Direct Red Flags

### From Context to Direct Signals

Previous features provided **context** (what's normal for this user/app/time).  
These features are **direct anomaly signals** based on domain expertise.

### Feature Categories

**1. Error-Based Indicators**
- `is_critical_error`: Flags `error_status == 2` (critical/severe errors)
- `has_error`: Flags any error (`error_status > 0`)

**Why Errors Matter:**
- Normal operations rarely produce errors
- Critical errors often indicate attacks, misconfigurations, or system compromise
- Error patterns correlate strongly with malicious activity



**2. Frequency-Based Indicators**

| Feature | Threshold | What It Catches |
|---------|-----------|----------------|
| `is_high_frequency` | > 100 | Extreme activity bursts |
| `is_very_high_frequency` | > 50 | Elevated activity |
| `frequency_zscore` | Z-score | Statistical outliers |

**Suspicious High-Frequency Patterns:**
- **DDoS attacks**: Flooding servers with requests
- **Data scraping**: Automated harvesting of information
- **Brute force**: Rapid password/key attempts
- **Data exfiltration**: Bulk data downloads

**Z-Score Explanation:**
- Z-score shows how many standard deviations from the mean
- Z > 3: Extreme outlier (99.7% of data is within ±3σ)
- Z > 10: Almost certainly anomalous
- Provides continuous measure vs. binary threshold



**3. Geographic/Behavioral Red Flags**

| Feature | What It Flags | Security Implication |
|---------|--------------|---------------------|
| `is_multi_country_user` | User accessing from >1 country | Account compromise, credential sharing |
| `is_invalid_asn` | ASN = 0 | Data tampering, spoofed network info |

**Why Multi-Country Access Is Suspicious:**
- Most users operate from 1 country (or 2 if traveling)
- Simultaneous multi-country access is physically impossible for one person
- Common in account takeover scenarios
- Credential sharing or VPN hopping

**Invalid ASN:**
- ASN (Autonomous System Number) identifies network providers
- ASN = 0 is technically invalid
- Might indicate spoofing, data errors, or intentional obfuscation



### Real-World Attack Example

A truly suspicious record might show:

- is_critical_error: 1
- has_error: 1
- is_very_high_frequency: 1  
- frequency_zscore: 8.5 (extreme!)
- is_multi_country_user: 1
- is_invalid_asn: 1
- is_night_hours: 1
- is_rare_browser: 1


**8 simultaneous red flags** → Almost certainly an attack or system compromise

### Creation of Anomaly Indicators

In [19]:
# Critical error indicator
df['is_critical_error'] = (df['error_status'] == 2).astype(int)
# Any error indicator
df['has_error'] = (df['error_status'] > 0).astype(int)
# High frequency indicator (> 100)
df['is_high_frequency'] = (df['frequency'] > 100).astype(int)
# Very high frequency indicator (> 50)
df['is_very_high_frequency'] = (df['frequency'] > 50).astype(int)
# Frequency z-score
df['frequency_zscore'] = (df['frequency'] - df['frequency'].mean()) / df['frequency'].std()
# Multi-country user indicator
df['is_multi_country_user'] = (df['user_country_count'] > 1).astype(int)
# ASN = 0 indicator (potentially invalid)
df['is_invalid_asn'] = (df['asn'] == 0).astype(int)

---
## 8. Categorical Feature Encoding

### Cardinality Analysis

Before encoding, let's analyze the cardinality (number of unique values) of each categorical feature to determine the appropriate encoding strategy.

**Purpose of This Analysis:**

Before encoding categorical features, we analyze cardinality to choose the right encoding strategy:

This ensures we don't create thousands of sparse columns while preserving important categorical information.

In [20]:
categorical_features = ['wf_principal_id', 'app_id', 'browser', 'isp', 'operation',
                        'log_type', 'atype', 'rw', 'hour_category', 'device_os', 'country']

# Calculate cardinality for each feature
cardinality_data = []

for col in categorical_features:
    unique_count = df[col].nunique()
    total_count = df[col].notna().sum()
    missing_count = df[col].isna().sum()
    missing_pct = (missing_count / len(df) * 100)
    
    cardinality_data.append({
        'Feature': col,
        'Unique_Values': unique_count,
        'Non_Missing': total_count,
        'Missing': missing_count,
        'Missing_%': f"{missing_pct:.1f}%"
    })

# Create DataFrame and sort by cardinality
cardinality_df = pd.DataFrame(cardinality_data)
cardinality_df = cardinality_df.sort_values('Unique_Values', ascending=False)

print("\nCATEGORICAL FEATURE CARDINALITY BREAKDOWN:")
print("-"*80)
print(cardinality_df.to_string(index=False))



CATEGORICAL FEATURE CARDINALITY BREAKDOWN:
--------------------------------------------------------------------------------
        Feature  Unique_Values  Non_Missing  Missing Missing_%
      operation            251        11261     4336     27.8%
         app_id            153        15597        0      0.0%
            isp             40        15555       42      0.3%
        browser             30        12121     3476     22.3%
wf_principal_id             19        15532       65      0.4%
        country              6         4337    11260     72.2%
       log_type              5        15597        0      0.0%
  hour_category              4        15597        0      0.0%
      device_os              4         5297    10300     66.0%
          atype              2        15597        0      0.0%
             rw              2        11123     4474     28.7%


### Encoding Approach

The below code uses **two different encoding strategies** based on feature characteristics:

### **1. Frequency Encoding** (High-Cardinality Features)
Applied to: `wf_principal_id`, `app_id`, `browser`, `isp`, `operation`

- Counts how many times each value appears in the dataset
- Replaces the categorical value with its count
- Example: If "Chrome" appears 5,000 times, all Chrome entries get encoded as 5000

### **2. One-Hot Encoding** (Low-Cardinality Features)
Applied to: `log_type`, `atype`, `rw`, `hour_category`, `device_os`, `country`

- Creates separate binary columns for each category
- Example: `log_type` with values [A, B, C] becomes 3 columns: `logtype_A`, `logtype_B`, `logtype_C`
- Each row has 1 in the relevant column, 0 in others

### Why Specific Methods for Specific Features

### **Frequency Encoding for High-Cardinality Features**

**High-cardinality** = Many unique values (hundreds/thousands)

| Feature | Why Frequency Encoding? |
|---------|------------------------|
| `wf_principal_id` | Could have 1000+ unique users - one-hot would create 1000+ columns! |
| `app_id` | Many different applications |
| `browser` | Dozens of browser types |
| `isp` | Hundreds of ISPs |
| `operation` | Many different operations |

**Advantages:**
- **Compact**: Only 1 column instead of 1000+
- **Preserves information**: Popular values get higher numbers, rare ones get lower numbers
- **Aligns with anomaly detection**: Rare categories (low frequency) are often anomalous!

**Example:**

- Chrome (5000 occurrences) → 5000
- Firefox (2000 occurrences) → 2000
- Rare_Browser_X (2 occurrences) → 2  ← Low number flags rarity!


### **One-Hot Encoding for Low-Cardinality Features**

**Low-cardinality** = Few unique values (2-10)

| Feature | # Categories | Why One-Hot? |
|---------|-------------|--------------|
| `log_type` | ~3-5 | Each type has distinct meaning |
| `atype` | ~3-5 | Authentication methods are qualitatively different |
| `rw` | 3 (read/write/unknown) | Clear categorical distinction |
| `hour_category` | 4 (morning/afternoon/evening/night) | Time periods have different risk profiles |
| `device_os` | 5-7 (top OS + other) | Operating systems behave differently |
| `country` | 4 (IN/US/other/missing) | Geographic regions have different patterns |

**Advantages:**
- **Preserves categorical distinctions**: "Morning" isn't "2x better" than "Evening" - they're just different
- **Model flexibility**: Allows the model to learn different patterns for each category
- **No ordinal assumption**: Doesn't imply any ordering or magnitude

**Example:**
```
hour_category = "morning" → hour_cat_afternoon=0, hour_cat_evening=0, hour_cat_morning=1, hour_cat_night=0
```

### Critical Importance for ML Models

### **1. Machine Learning Requires Numbers**
- Text like "Chrome" or "morning" must be converted to numbers

### **2. The Curse of Dimensionality**
If we used one-hot encoding for `wf_principal_id` with 1000 users:
- Creates 1000 new columns
- Dataset becomes sparse (mostly zeros)
- Model training becomes slow and unstable
- Risk of overfitting increases dramatically

**Frequency encoding avoids this** by keeping it as 1 column.

### **3. Semantic Meaning Preservation**

**Frequency encoding captures "popularity":**
- High frequency = common, likely normal
- Low frequency = rare, potentially anomalous
- This is exactly what we want for anomaly detection!

**One-hot encoding captures "identity":**
- Each category is treated as equally distinct
- Model learns: "When logtype_A=1, behavior X is normal"
- Different from: "When logtype_B=1, behavior Y is normal"

### Special Handling Details

### **Device OS - Top 5 + "Other"**
```python
top_os = df['device_os'].value_counts().head(5).index.tolist()
df['device_os_filled'] = df['device_os'].apply(lambda x: x if x in top_os else 'other')
```
**Why?** Balance between detail and dimensionality
- Keep the 5 most common OS (Windows, Mac, Android, iOS, Linux)
- Group rare ones as "other"
- Prevents dozens of rare OS categories from creating noise

### **Country - Simplified to 4 Groups**
```python
df['country_grouped'] = df['country'].apply(
    lambda x: x if x in ['IN', 'US'] else ('other' if pd.notna(x) else 'missing')
)
```
**Why?** 
- IN (India) and US have more entries, while others have only one
- Groups uncommon countries
- Explicitly handles missing values

### **RW - Filling Unknown**
```python
df['rw_filled'] = df['rw'].fillna('unknown')
```
**Why?** Missing read/write indicator is itself informative - some operations might not have this field, which could be suspicious

### Real-World Impact

**Without proper encoding:**
- Model can't run (text data)
- Information is lost

**With frequency encoding only:**
- Works but loses categorical distinctions
- "Morning" as 1, "Afternoon" as 2 implies afternoon is "more than" morning

**With one-hot encoding only:**
- Works but creates 1000+ columns for user_id
- Slow, memory-intensive, unstable

**With this hybrid approach:**
- Compact representation
- Preserves rarity information (frequency encoding)
- Preserves categorical distinctions (one-hot encoding)
- Optimal balance for anomaly detection

**Bottom line**: This encoding strategy is carefully designed to give the model maximum information with minimum complexity, specifically optimized for detecting anomalous patterns in log data.

In [21]:
# Frequency encoding for high-cardinality features
# User ID frequency encoding
wf_principal_freq = df['wf_principal_id'].value_counts().to_dict()
df['wf_principal_id_freq_encoded'] = df['wf_principal_id'].map(wf_principal_freq).fillna(0)

# App ID frequency encoding
app_freq = df['app_id'].value_counts().to_dict()
df['app_id_freq_encoded'] = df['app_id'].map(app_freq)

# Browser frequency encoding
browser_freq = df['browser'].value_counts().to_dict()
df['browser_freq_encoded'] = df['browser'].map(browser_freq).fillna(0)

# ISP frequency encoding
isp_freq = df['isp'].value_counts().to_dict()
df['isp_freq_encoded'] = df['isp'].map(isp_freq).fillna(0)

# Operation frequency encoding
op_freq = df['operation'].value_counts().to_dict()
df['operation_freq_encoded'] = df['operation'].map(op_freq).fillna(0)

# One-hot encoding for low-cardinality features (using drop_first=True to avoid dummy variable trap)
# Log type
log_type_dummies = pd.get_dummies(df['log_type'], prefix='logtype', drop_first=True, dtype=int)
df = pd.concat([df, log_type_dummies], axis=1)

# Authentication type
atype_dummies = pd.get_dummies(df['atype'], prefix='atype', drop_first=True, dtype=int)
df = pd.concat([df, atype_dummies], axis=1)

# Read/Write operation
df['rw_filled'] = df['rw'].fillna('unknown')
rw_dummies = pd.get_dummies(df['rw_filled'], prefix='rw', drop_first=True, dtype=int)
df = pd.concat([df, rw_dummies], axis=1)

# Hour category
hour_cat_dummies = pd.get_dummies(df['hour_category'], prefix='hour_cat', drop_first=True, dtype=int)
df = pd.concat([df, hour_cat_dummies], axis=1)

# Device OS (top categories)
if df['device_os'].notna().sum() > 0:
    top_os = df['device_os'].value_counts().head(5).index.tolist()
    df['device_os_filled'] = df['device_os'].apply(lambda x: x if x in top_os else 'other').fillna('missing')
    os_dummies = pd.get_dummies(df['device_os_filled'], prefix='os', drop_first=True, dtype=int)
    df = pd.concat([df, os_dummies], axis=1)

# Country (simplified to IN, US, Other, Missing)
df['country_grouped'] = df['country'].apply(
    lambda x: x if x in ['IN', 'US'] else ('other' if pd.notna(x) else 'missing')
)
country_dummies = pd.get_dummies(df['country_grouped'], prefix='country', drop_first=True, dtype=int)
df = pd.concat([df, country_dummies], axis=1)


### Implementation Details

**Frequency Encoding Process:**
1. Count occurrences: `value_counts().to_dict()`
2. Map counts back to original column: `.map()`
3. Fill NaN with 0 (for missing values)
4. Result: Each category replaced by its frequency

**One-Hot Encoding Process:**
1. Create binary columns: `pd.get_dummies(drop_first=True)`
2. Use meaningful prefixes: `prefix='logtype'`
3. **Drop first category** to avoid dummy variable trap (n categories → n-1 features)
4. Concatenate to dataframe: `pd.concat()`
5. Result: N categories → N-1 binary columns

**Why drop_first=True?**
- **Prevents multicollinearity**: When all other categories are 0, the dropped category is implied
- **Reduces dimensionality**: For 4 hour categories, we only need 3 binary features instead of 4
- **More efficient**: Fewer features = faster training, less memory
- **Example**: If `hour_cat_morning=0, hour_cat_afternoon=0, hour_cat_evening=0`, then it must be `night`

**Special Handling:**
- **RW field**: Fill missing with 'unknown' before encoding (missingness is informative)
- **Device OS**: Keep only top 5, group rest as 'other' (balance detail vs. complexity)
- **Country**: Simplify to IN/US/other/missing (focus on key markets)

---

## 9. Handling Remaining Missing Values in Numeric Features

### Why Fill Numeric Missing Values?

After creating missing value **indicators**, we still need to fill the actual missing values because:
- Machine learning models can't process NaN values
- We've already preserved the "missingness" information via indicator features
- Filling allows us to use all records without dropping data

### Filling Strategy

| Feature | Fill Value | Rationale |
|---------|-----------|-----------|
| `asn` | 0 | Represents "unknown network" - captured by `is_invalid_asn` feature |
| `device_os_version` | 0 | Represents "unknown version" - less critical than OS itself |
| `uatype` | 0 | Represents "unknown type" - missingness already captured |

**Note**: We use 0 rather than median/mean because:
- We already flagged missingness with indicator features
- 0 is a safe neutral value that won't skew statistics
- It represents a distinct "unknown" category

In [22]:
# Fill numerical nulls with 0
df['asn'] = df['asn'].fillna(0)
df['device_os_version'] = df['device_os_version'].fillna(0)
df['uatype'] = df['uatype'].fillna(0)

---

## 10. Feature Scaling with RobustScaler

### Why Scaling Matters for Anomaly Detection

**The Problem:**
Distance-based algorithms (LOF, DBSCAN, KNN) calculate distances between points. Features with larger ranges dominate these calculations.

**Example:**
- Feature A: frequency (range: 1 - 10,000)
- Feature B: hour (range: 0 - 23)

Without scaling, `frequency` differences overwhelm `hour` differences in distance calculations, even though hour might be equally important for detecting anomalies.


### Scaling Method Comparison

| Method | How It Works | Outlier Sensitive? | Good for Anomaly Detection? |
|--------|-------------|-------------------|----------------------------|
| **StandardScaler** | (X - mean) / std | YES | Anomalies corrupt mean/std |
| **MinMaxScaler** | (X - min) / (max - min) | VERY | Extreme values compress normal range |
| **RobustScaler** | **(X - median) / IQR** | **NO** | **BEST - Uses robust statistics** |
| **MaxAbsScaler** | X / max(abs(X)) | YES | Affected by extreme values |


### Why RobustScaler is Superior

**1. Outlier-Resistant Statistics**
- Uses **median** (50th percentile) instead of mean
  - Median is unaffected by extreme values
  - Example: [1, 2, 3, 4, 100] → median = 3, mean = 22

- Uses **IQR** (Interquartile Range = Q3 - Q1) instead of standard deviation
  - Only considers middle 50% of data (25th to 75th percentile)
  - Extreme values don't affect the scale

**2. Preserves Anomaly Characteristics**
- Normal data scales to approximately [-1, 1]
- Anomalies remain far from center (large absolute values)
- The "outlierness" we want to detect is preserved!


### Implementation Strategy

**Features to Scale:**
- All numerical features (frequency, hour, day, etc.)
- User/app aggregation statistics
- Frequency-encoded categorical features

**Features NOT to Scale:**
- Binary indicators (already 0/1)
- One-hot encoded features (already 0/1)

**Process:**
1. Fit RobustScaler on each numerical feature
2. Transform to scaled values
3. Create new columns with `_scaled` suffix
4. Preserve original values for interpretability

In [23]:
# Select features to scale
numerical_features_to_scale = [
    'frequency', 'frequency_zscore', 'asn', 'uatype', 'device_os_version',
    'hour', 'day_of_week', 'day_of_month',
    'user_total_activity', 'user_error_rate', 'user_country_count', 'user_asn_count',
    'user_browser_count', 'user_total_frequency', 'user_avg_frequency', 'user_max_frequency',
    'app_total_activity', 'app_error_rate', 'app_total_errors',
    'app_total_frequency', 'app_avg_frequency', 'app_max_frequency',
    'user_id_freq_encoded', 'app_id_freq_encoded', 'browser_freq_encoded',
    'isp_freq_encoded', 'operation_freq_encoded'
]

# Use RobustScaler (less sensitive to outliers)
scaler = RobustScaler()

for feature in numerical_features_to_scale:
    if feature in df.columns:
        df[f'{feature}_scaled'] = scaler.fit_transform(df[[feature]])


---

## 11. Final Feature Selection and Dataset Preparation

### Feature Selection Strategy

Selecting features based on three criteria:

1. **Relevance to Anomaly Detection**
   - Direct anomaly signals (errors, high frequency, rarity)
   - Behavioral context (user/app statistics)
   - Temporal patterns (time-based indicators)

2. **Feature Type Diversity**
   - Binary indicators: Simple yes/no flags
   - Scaled numerical: Continuous measures
   - One-hot encoded: Categorical distinctions

3. **Avoiding Redundancy**
   - Not including both raw AND scaled versions
   - Using scaled versions for modeling
   - Keeping original values for interpretation only


### Feature Categories in Final Set

**1. Binary Indicators (Flags)**
- Missing value indicators: `has_user_id`, `has_browser`, `has_device_os`, etc.
- Temporal flags: `is_weekend`, `is_business_hours`, `is_night_hours`
- Rarity flags: `is_rare_operation`, `is_rare_browser`, `is_rare_isp`, etc.
- Anomaly signals: `is_critical_error`, `has_error`, `is_high_frequency`, etc.

**2. Scaled Numerical Features**
- Activity metrics: `frequency_scaled`, `user_total_activity_scaled`
- Network info: `asn_scaled`
- Temporal: `hour_scaled`, `day_of_week_scaled`
- Aggregations: `user_error_rate_scaled`, `app_error_rate_scaled`
- Frequency-encoded: `user_id_freq_encoded_scaled`, `browser_freq_encoded_scaled`

**3. One-Hot Encoded Categories**
- Log types: `logtype_*`
- Authentication: `atype_*`
- Operations: `rw_*`
- Time periods: `hour_cat_*`
- Device: `os_*`
- Geography: `country_*`


In [24]:
# Define final feature set for modeling
model_features = []

# Binary indicators
model_features.extend([
    'has_user_id', 'has_browser', 'has_device_os', 'has_country', 'has_operation',
    'is_weekend', 'is_business_hours', 'is_night_hours',
    'is_rare_operation', 'is_rare_browser', 'is_rare_isp', 'is_rare_country', 'is_rare_user',
    'is_critical_error', 'has_error', 'is_high_frequency', 'is_very_high_frequency',
    'is_multi_country_user', 'is_invalid_asn'
])

# Scaled numerical features
model_features.extend([f'{f}_scaled' for f in [
    'frequency', 'asn', 'uatype', 'hour', 'day_of_week',
    'user_total_activity', 'user_error_rate', 'user_country_count',
    'app_total_activity', 'app_error_rate',
    'user_id_freq_encoded', 'app_id_freq_encoded', 'browser_freq_encoded',
    'isp_freq_encoded', 'operation_freq_encoded'
] if f'{f}_scaled' in df.columns])

# One-hot encoded features
model_features.extend(log_type_dummies.columns.tolist())
model_features.extend(atype_dummies.columns.tolist())
model_features.extend(rw_dummies.columns.tolist())
model_features.extend(hour_cat_dummies.columns.tolist())
if 'os_dummies' in locals():
    model_features.extend(os_dummies.columns.tolist())
model_features.extend(country_dummies.columns.tolist())

# Filter to only features that exist
model_features = [f for f in model_features if f in df.columns]

print(f"* Selected {len(model_features)} features for modeling")
print(f"\nFeature Categories:")
print(f"  - Binary indicators: {len([f for f in model_features if f.startswith('is_') or f.startswith('has_')])}")
print(f"  - Scaled numerical: {len([f for f in model_features if f.endswith('_scaled')])}")
print(f"  - One-hot encoded: {len([f for f in model_features if any(f.startswith(p) for p in ['logtype_', 'atype_', 'rw_', 'hour_cat_', 'os_', 'country_'])])}")

# Create final dataset with original_index for traceability
X = df[model_features].copy()
X['original_index'] = df['original_index']  # Add index column for anomaly tracing

print(f"Final feature matrix shape: {X.shape}")

* Selected 49 features for modeling

Feature Categories:
  - Binary indicators: 18
  - Scaled numerical: 14
  - One-hot encoded: 17
Final feature matrix shape: (15597, 50)


In [25]:
# Save the full engineered dataset
df.to_csv('data/engineered_features_full_data.csv', index=False)
print(f"* Saved full engineered dataset: data/engineered_features_full_data.csv")

# Save the model-ready feature matrix
X.to_csv('data/model_features_data.csv', index=False)
print(f"\n* Saved model-ready features: data/model_features_data.csv")

# Save feature names
with open('data/feature_names.txt', 'w') as f:
    for feature in model_features:
        f.write(f"{feature}\n")

* Saved full engineered dataset: data/engineered_features_full_data.csv

* Saved model-ready features: data/model_features_data.csv


<h1><center>END</center></h1>